# Climate Data Preprocessing

In [1]:
# extract climate data to put it in a pandas dataframe "data":
# lon: longitute > -10 to 30
# lat: latitude > 33 to 52
# year: year > 2001 to 2019
# month: august (7)
# tas: mean temperature 
# tasmax: maximum temperature
# pr: total precipitation
# each value for august from 2001 to 2019 and per location > 19*41*20 = 15580 rows

import xarray as xr
import numpy as np
import pandas as pd

data = pd.DataFrame({'lon': pd.Series(dtype='float64'),
                      'lat': pd.Series(dtype='float64'),
                      'year': pd.Series(dtype='int64'),
                      'month': pd.Series(dtype='int64'),
                      'tas': pd.Series(dtype='float32'),
                      'tasmax': pd.Series(dtype='float32'),
                      'pr': pd.Series(dtype='float32')})


for i in range(19):
    
    temp_mean = xr.open_dataset(f'climate/CRU_mean_temperature_mon_0.5x0.5_global_{2001+i}_v4.03.nc').to_dataframe()
    temp_max = xr.open_dataset(f'climate/CRU_maximum_temperature_mon_0.5x0.5_global_{2001+i}_v4.03.nc').to_dataframe()
    pre_tot = xr.open_dataset(f'climate/CRU_total_precipitation_mon_0.5x0.5_global_{2001+i}_v4.03.nc').to_dataframe()
    temp_mean.reset_index(inplace=True)
    temp_max.reset_index(inplace=True)
    pre_tot.reset_index(inplace=True)
    
    frame = pd.concat([temp_mean['lon'],temp_mean['lat'],temp_mean['time'],temp_mean['tas'],temp_max['tasmax'],pre_tot['pr']],axis=1)
    frame['month'] = frame['time'].dt.month
    frame['year'] = frame['time'].dt.year
    frame.drop('time', inplace=True, axis=1)  
    frame = frame[(frame['lat']>=32.75) & (frame['lat']<=52.25) & (frame['lon']>=-10.25) & (frame['lon']<=30.25) & (frame['month']==7)]
    frame['lon'] = frame['lon'].round(decimals=0)
    frame['lat'] = frame['lat'].round(decimals=0)
    frame = frame.groupby(['lon','lat','month','year']).agg({'tas':'mean','tasmax':'max','pr':'mean'})
    frame.reset_index(inplace=True)
    
    data = pd.concat([data,frame])

data.reset_index(inplace=True) 
data.drop('index', inplace=True, axis=1)  

data

FileNotFoundError: [Errno 2] No such file or directory: '/home/ademmanuel_01/code/TomasDemeter/FireProject/notebooks/climate/CRU_mean_temperature_mon_0.5x0.5_global_2001_v4.03.nc'